# 1.1.4.3: Ortogonalización y Estabilidad Numérica

## Objetivos de Aprendizaje

Al completar este notebook, serás capaz de:

- **Diagnosticar** la inestabilidad de una matriz usando el **Número de Condición**.
- **Entender** por qué las **bases ortonormales** son el "santo grial" para la estabilidad numérica en problemas de mínimos cuadrados.
- **Aplicar** el proceso de **Ortogonalización de Gram-Schmidt** para crear una base ortonormal a partir de una base cualquiera.
- **Utilizar** la **Descomposición QR** como la encarnación computacionalmente eficiente de Gram-Schmidt.
- **Resolver** problemas de Mínimos Cuadrados de forma robusta usando la Descomposición QR y comparar su estabilidad con el método de las Ecuaciones Normales.

In [ ]:
# --- Celda de Configuración (Oculta) ---
%display latex
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(style="whitegrid")

def plot_gram_schmidt(v1, v2):
    proj_v2_on_v1 = (v2.dot(v1) / v1.dot(v1)) * v1
    u2 = v2 - proj_v2_on_v1
    
    plt.figure(figsize=(8, 8))
    # Vectores originales
    plt.quiver(0, 0, v1[0], v1[1], angles='xy', scale_units='xy', scale=1, color='#0072B2', width=0.015, label='v1 (base original)')
    plt.quiver(0, 0, v2[0], v2[1], angles='xy', scale_units='xy', scale=1, color='#D55E00', width=0.015, label='v2 (base original)')
    # Proceso
    plt.quiver(0, 0, proj_v2_on_v1[0], proj_v2_on_v1[1], angles='xy', scale_units='xy', scale=1, color='gray', linestyle='--', label='proy_v1(v2)')
    plt.quiver(proj_v2_on_v1[0], proj_v2_on_v1[1], u2[0], u2[1], angles='xy', scale_units='xy', scale=1, color='#009E73', width=0.015, label='u2 (componente ortogonal)')
    
    limit = np.max(np.abs(np.vstack((v1, v2, u2)))) * 1.2
    plt.xlim(-limit, limit); plt.ylim(-limit, limit)
    plt.grid(True); plt.legend(); plt.title('Visualización del Proceso de Gram-Schmidt')
    plt.gca().set_aspect('equal', adjustable='box')
    plt.show()

--- 
## ⚙️ El Arsenal de Datasets: Nuestra Fuente de Ejercicios

La estabilidad numérica es un problema del mundo real. Para estudiarlo, necesitamos datasets que imiten los problemas que encontramos en la práctica, como la multicolinealidad. También usaremos vectores simples para visualizar la geometría de la ortogonalización.

In [ ]:
# === CONFIGURACIÓN DE DATASETS ===
from src.data_generation.create_student_performance import create_student_performance_data
from src.data_generation.create_edge_cases import create_edge_cases

# Configuración centralizada de aleatoriedad para REPRODUCIBILIDAD
rng = np.random.default_rng(seed=42)

# === Generación de Datasets y Matrices para este Notebook ===

# 💡 CONTEXTO PEDAGÓGICO: Hilo Conductor (Regresión Robusta)
# Usaremos el dataset de estudiantes para comparar la solución de OLS obtenida con las
# Ecuaciones Normales vs. la obtenida con la Descomposición QR, demostrando que en
# casos bien condicionados, ambas coinciden.
datos_estudiantes = create_student_performance_data(rng, simplified=True, n_samples=100)

# 💡 CONTEXTO PEDAGÓGICO: El Caso Problema (Multicolinealidad)
# La mejor forma de apreciar la estabilidad de QR es usarlo donde las Ecuaciones Normales
# sufren. Este dataset con alta multicolinealidad será nuestro campo de pruebas.
datos_multicolineales = create_edge_cases(rng, case_type='multicollinear', n_samples=100)

# 💡 CONTEXTO PEDAGÓGICO: Vectores para Geometría
# Para visualizar Gram-Schmidt, necesitamos vectores que NO sean ortogonales, para que
# el algoritmo tenga algo que corregir.
v1 = np.array([3, 1])
v2 = np.array([2, 2])

print("Datasets y vectores generados y listos para usar.")

## 1. El Problema: Cuando las Ecuaciones Normales son Inestables

En el notebook anterior, derivamos las Ecuaciones Normales $X^T X \hat{\beta} = X^T \vec{y}$. Esta fórmula es teóricamente correcta, pero tiene un talón de Aquiles numérico: la matriz $X^T X$.

Si las columnas de $X$ son casi linealmente dependientes (es decir, hay **multicolinealidad**), la matriz $X^T X$ se vuelve **mal condicionada**. Esto significa que es muy sensible a pequeños errores de redondeo en los cálculos de punto flotante, y la solución $\hat{\beta}$ puede ser muy imprecisa.

#### Diagnóstico: El Número de Condición
El **Número de Condición** de una matriz, `np.linalg.cond(A)`, mide esta sensibilidad. Un número cercano a 1 es ideal (como en una matriz ortogonal). Un número muy grande (e.g., > 1,000) es una bandera roja 🚩 de inestabilidad.

**Regla Clave:** `cond(XᵀX) = cond(X)²`. Formar la matriz $X^T X$ **eleva al cuadrado el número de condición**, empeorando drásticamente la estabilidad del problema. Por esto, los métodos modernos evitan formar $X^T X$ explícitamente.

### Ejemplo Demostrativo 1: El Número de Condición Explota

In [ ]:
# 1. DATOS: Usamos el dataset con multicolinealidad, donde x2 y x3 dependen de x1.
X_mal_condicionado = datos_multicolineales[['x1', 'x2', 'x3']].values
X_bien_condicionado = datos_estudiantes[['horas_estudio', 'calificacion_examen']].values

# 2. APLICACIÓN: Comparamos los números de condición.
cond_X_bien = np.linalg.cond(X_bien_condicionado)
cond_XTX_bien = np.linalg.cond(X_bien_condicionado.T @ X_bien_condicionado)

cond_X_mal = np.linalg.cond(X_mal_condicionado)
cond_XTX_mal = np.linalg.cond(X_mal_condicionado.T @ X_mal_condicionado)

# 3. INTERPRETACIÓN
print("--- Caso Bien Condicionado ---")
print(f"Número de Condición de X: {cond_X_bien:.2f}")
print(f"Número de Condición de XᵀX: {cond_XTX_bien:.2f}")

print("\n--- Caso Mal Condicionado (Multicolinealidad) ---")
print(f"Número de Condición de X: {cond_X_mal:,.2f}")
print(f"Número de Condición de XᵀX: {cond_XTX_mal:,.2f} (¡Inmanejablemente alto!)")

print(f"\nObserva que {cond_X_mal**2:,.2f} es aproximadamente igual a {cond_XTX_mal:,.2f}.")

## 2. La Solución: Ortogonalización

El problema es que las columnas de $X$ "se parecen" entre sí. La solución es encontrar una **base ortonormal** para el espacio columna de $X$. Una base ortonormal está compuesta por vectores que son:
1.  **Ortogonales:** Todos son perpendiculares entre sí (producto punto es cero).
2.  **Normales:** Todos tienen longitud 1 (norma L2 es uno).

Si pudiéramos reemplazar nuestra matriz $X$ por una matriz $Q$ con columnas ortonormales, las Ecuaciones Normales se simplificarían a $I \hat{\beta} = Q^T \vec{y}$, una solución trivial y perfectamente estable.

### El Algoritmo: Ortogonalización de Gram-Schmidt
Este es el algoritmo clásico para construir una base ortogonal $\{\vec{u_1}, \vec{u_2}, \dots\}$ a partir de una base cualquiera $\{\vec{v_1}, \vec{v_2}, \dots\}$.
- **Paso 1:** $\vec{u_1} = \vec{v_1}$
- **Paso 2:** Se toma $\vec{v_2}$ y se le resta su proyección sobre $\vec{u_1}$. Lo que queda es ortogonal a $\vec{u_1}$.
  $$ \vec{u_2} = \vec{v_2} - \text{proj}_{\vec{u_1}}(\vec{v_2}) $$
- **Paso 3 (y siguientes):** Para $\vec{v_k}$, se le restan sus proyecciones sobre todos los vectores ortogonales ya encontrados ($\{\vec{u_1}, \dots, \vec{u_{k-1}}\}$).
- **Normalización Final:** Se divide cada vector $\vec{u_i}$ por su norma para obtener una base ortonormal.

### Ejemplo Demostrativo 2: Visualización de Gram-Schmidt

In [ ]:
# 1. DATOS: Dos vectores linealmente independientes pero no ortogonales.
v1 = np.array([3, 1])
v2 = np.array([2, 2])
print(f"Producto punto original (v1·v2): {v1 @ v2} (No es cero)")

# 2. APLICACIÓN: Aplicamos el proceso
u1 = v1
proj_v2_on_u1 = (v2 @ u1) / (u1 @ u1) * u1
u2 = v2 - proj_v2_on_u1

# 3. INTERPRETACIÓN Y VERIFICACIÓN
print(f"Nueva base ortogonal: u1={np.round(u1, 2)}, u2={np.round(u2, 2)}")
print(f"Producto punto nuevo (u1·u2): {u1 @ u2:.10f} (Es cero)")

# 4. VISUALIZACIÓN
plot_gram_schmidt(v1, v2)

## 3. La Herramienta Práctica: Descomposición QR

En la práctica, no implementamos Gram-Schmidt a mano (es numéricamente inestable para muchas columnas). Usamos la **Descomposición QR**, que es la encarnación de este proceso, implementada de forma robusta. Descompone cualquier matriz $A$ de $m \times n$ en el producto:
$$ A = QR $$
- **$Q$**: Una matriz $m \times n$ con **columnas ortonormales** que forman una base para el Espacio Columna de A.
- **$R$**: Una **matriz triangular superior** $n \times n$ que "registra" cómo se combinaron los vectores de Q para reconstruir A.

### Resolviendo Mínimos Cuadrados con QR
Sustituimos $X=QR$ en el problema de mínimos cuadrados $X\vec{\beta} = \vec{y}$:
$$ QR\vec{\beta} = \vec{y} $$
Multiplicamos por $Q^T$. Como $Q^T Q = I$, obtenemos:
$$ R\hat{\beta} = Q^T\vec{y} $$
Como R es triangular, este sistema es muy fácil y **numéricamente estable** de resolver para $\hat{\beta}$ usando un método llamado "sustitución hacia atrás".

### Ejemplo Demostrativo 3: Comparando OLS: Ecuaciones Normales vs. Descomposición QR

In [ ]:
# 1. DATOS: Usaremos el dataset con alta multicolinealidad.
X = datos_multicolineales[['x1', 'x2', 'x3']].values
y = datos_multicolineales['y'].values

# 2. APLICACIÓN: Resolvemos de ambas formas.

# Método 1: Ecuaciones Normales (Potencialmente Inestable)
XTX = X.T @ X
XTy = X.T @ y
try:
    beta_norm = np.linalg.solve(XTX, XTy)
except np.linalg.LinAlgError:
    beta_norm = [np.nan, np.nan, np.nan] # Falló

# Método 2: Descomposición QR (Robusto)
Q, R = np.linalg.qr(X)
QTy = Q.T @ y
beta_qr = np.linalg.solve(R, QTy)

# 3. INTERPRETACIÓN Y COMPARACIÓN
print(f"Número de Condición de XᵀX: {np.linalg.cond(XTX):,.2f}")
print(f"\nSolución con Ecuaciones Normales: {np.round(beta_norm, 4)}")
print(f"Solución con Descomposición QR: {np.round(beta_qr, 4)}")
print("\nConclusión: La descomposición QR proporciona una solución estable incluso cuando las Ecuaciones Normales fallan o son imprecisas.")

---
## 4. Ejercicios Guiados con Scaffolding (8+)
Rellena las partes marcadas con `# COMPLETAR` para afianzar tu comprensión.

### === EJERCICIO GUIADO 1: Un Paso de Gram-Schmidt ===

In [ ]:
# DATOS
v1 = np.array([1, 1, 0])
v2 = np.array([1, 3, 1])

# Nuestro primer vector ortogonal es v1.
u1 = v1

# TODO 1: Calcula la proyección de v2 sobre u1.
proj_v2_on_u1 = # COMPLETAR

# TODO 2: Calcula el segundo vector ortogonal, u2, restando la proyección de v2.
u2 = # COMPLETAR

# VERIFICACIÓN
assert np.allclose(proj_v2_on_u1, np.array([2, 2, 0]))
assert np.isclose(u1 @ u2, 0), "Los vectores u1 y u2 no son ortogonales."
print("✅ ¡Paso de Gram-Schmidt completado con éxito!")
print(f"El componente de v2 ortogonal a v1 es: {u2}")

### === EJERCICIO GUIADO 2: Normalización ===

In [ ]:
# DATOS: El vector u2 del ejercicio anterior.
u2 = np.array([-1, 1, 1])

# TODO: Normaliza el vector u2 (divídelo por su norma L2).
e2 = # COMPLETAR

# VERIFICACIÓN
assert np.isclose(np.linalg.norm(e2), 1.0), "El vector no tiene norma 1."
print("✅ ¡Vector normalizado correctamente!")
print(f"Vector unitario e2: {np.round(e2, 3)}")

### === EJERCICIO GUIADO 3: Realizar la Descomposición QR ===

In [ ]:
# DATOS
A = np.array([[1, 1], [1, 2], [1, 3]])

# TODO: Usa np.linalg.qr() para descomponer A en Q y R.
Q, R = # COMPLETAR

# VERIFICACIÓN
assert Q.shape == (3, 2) and R.shape == (2, 2), "Las formas de Q y R son incorrectas."
assert np.allclose(Q.T @ Q, np.identity(2)), "Las columnas de Q no son ortonormales."
assert np.allclose(A, Q @ R), "La reconstrucción Q@R no es igual a A."
print("✅ ¡Descomposición QR exitosa!")
print(f"Matriz Q (columnas ortonormales):\n{np.round(Q, 3)}")
print(f"\nMatriz R (triangular superior):\n{np.round(R, 3)}")

### === EJERCICIO GUIADO 4: Resolver Mínimos Cuadrados con QR ===

In [ ]:
# DATOS
X = np.array([[1, 1], [1, 2], [1, 3]])
y = np.array([1, 2, 2])

# Ya tenemos Q y R de X del ejercicio anterior.
Q, R = np.linalg.qr(X)

# El problema a resolver es R·β = Qᵀ·y

# TODO 1: Calcula el lado derecho de la ecuación, Qᵀ·y.
QTy = # COMPLETAR

# TODO 2: Resuelve el sistema triangular para encontrar beta.
beta_hat_qr = # COMPLETAR

# VERIFICACIÓN
beta_hat_norm = np.linalg.solve(X.T @ X, X.T @ y)
assert np.allclose(beta_hat_qr, beta_hat_norm)
print("✅ ¡Sistema resuelto con QR!")
print(f"β_hat = {np.round(beta_hat_qr, 2)}")

--- 
# 5. Banco de Ejercicios Prácticos (30+)
Ahora te toca a ti. Resuelve estos ejercicios para consolidar tu conocimiento.

### Parte A: Estabilidad y Gram-Schmidt

**A1 (🟢 Fácil):** Calcula el número de condición de $A = \begin{pmatrix} 1 & 2 \\ 3 & 4 \end{pmatrix}$ y de $A^TA$.

**A2 (🟢 Fácil):** Aplica el primer paso de Gram-Schmidt (encontrar $\vec{u_2}$) a los vectores $\vec{v_1} = [1, 1]$ y $\vec{v_2} = [0, 2]$.

**A3 (🟡 Medio):** Completa el proceso de Gram-Schmidt para los vectores del ejercicio A2 para encontrar una base ortonormal $\{\vec{e_1}, \vec{e_2}\}$.

**A4 (🔴 Reto):** Aplica el proceso de Gram-Schmidt a los 3 vectores: $\vec{v_1}=[1,1,0], \vec{v_2}=[1,0,1], \vec{v_3}=[0,1,1]$.

### Parte B: Descomposición QR

**B1 (🟢 Fácil):** Calcula la descomposición QR de la matriz $A = \begin{pmatrix} 1 & 2 \\ 0 & 3 \end{pmatrix}$.

**B2 (🟢 Fácil):** Verifica que para la matriz Q del ejercicio B1, se cumple que $Q^T Q$ es (aproximadamente) la matriz identidad.

**B3 (🟡 Medio):** Usando `datos_estudiantes`, construye la matriz de diseño $X$ y calcula su descomposición QR.

**B4 (🔴 Reto):** Resuelve el problema de regresión del **Hilo Conductor** usando la Descomposición QR. Los pasos son:
1. Construye la matriz de diseño `X` y el vector `y`.
2. Calcula `Q, R = np.linalg.qr(X)`.
3. Calcula `Qty = Q.T @ y`.
4. Resuelve el sistema triangular `R @ beta = Qty` para `beta` usando `np.linalg.solve`.
5. Compara los coeficientes con los obtenidos con las Ecuaciones Normales en el notebook anterior.

---

## ✅ Mini-Quiz de Autoevaluación

*Responde estas preguntas para verificar tu comprensión.*

1. ¿Por qué es problemático usar las Ecuaciones Normales cuando una matriz tiene un alto número de condición?
2. ¿Qué propiedad clave tienen las columnas de la matriz Q en una descomposición QR?
3. ¿Qué algoritmo teórico es la base de la descomposición QR?
4. Al resolver $R\hat{\beta} = Q^T\vec{y}$, ¿por qué es computacionalmente "fácil" encontrar $\hat{\beta}$?

## 🚀 Próximos Pasos

Has aprendido sobre la importancia de la estabilidad numérica y cómo la ortogonalización nos proporciona métodos más robustos para resolver problemas de la vida real. Este es el estándar de oro para resolver problemas de mínimos cuadrados en software profesional.

- En el notebook **`1.1.5.1_Descomposicion_de_Valor_Singular_SVD.ipynb`**, exploraremos la SVD, la "navaja suiza" de las descomposiciones matriciales. Es el método más robusto y general de todos, capaz de resolver sistemas de mínimos cuadrados incluso cuando las columnas de X son linealmente dependientes.